In [ ]:
from viewscad import Renderer

r = Renderer()
r.width = 700
r.height = 700

render = r.render


In [ ]:
from stackable_pipe import ring, connection, pipe
from solid import cylinder, translate, rotate, cube
from viewscad import Renderer

r = Renderer()
r.width = 700
r.height = 700

render = r.render


def bottom_grid(d=60, w=2, hole_size=6, hole_gap=2):
    plate = cylinder(h=w, d=d, segments=100)
    hole = cylinder(h=w, d=hole_size, segments=10)
    pane = range(-d//2, d//2, hole_size+hole_gap)
    transition = ring(d1=d, h=w, w=w*2)

    holes = [translate((x, y, 0))(hole) for x in pane for y in pane]

    return plate - holes + transition


def spin_around_tower(obj, count=4, alpha=45, h=160):
    angles = range(180, 540, int(360/count))
    # drehen und dann kippen
    return translate((0, 0, h/2))([rotate((alpha, 0, z))(obj) for z in angles])


def cup_con(d1=52, d2=60, w=2, height=80, inlet_delta=5):
    c = d2
    gradient = (d2-d1)/height
    inlet = (gradient*inlet_delta+w) * 2
    socket_d1_delta = d1 + inlet
    socket_d2_delta = d2 + inlet
    cutout = cube(c)\
        - translate((c/2, 0, 0))(cylinder(r=c/2, h=c, segments=100))\
        - translate((c/2, c, 0))(cylinder(r=c/2, h=c, segments=100))\
        - translate((c/2, 0, 0))(cube(c))

    cutout = translate((c/2, -c/2, 0))(rotate((0, -90, 0))(cutout))
    cutout = rotate(90)(cutout)
    cutout = translate((0, c/2, 0))(cutout)

    base = ring(d1=socket_d1_delta, d2=socket_d2_delta, h=height, w=w)
    socket = base - cutout


    cup = ring(d1=d1, d2=d2, w=w, h=height)
    cup += bottom_grid(d=d1, w=w)

    outline = cylinder(d1=d1, d2=d2, h=height*2, segments=100)

    return (socket, cup, outline)


def aero_tower(w=2, h=160, d=160, socket_h=20, pipe_count=1, alpha=45, cup_d = 60, cup_h=80, tooth=2):
    gap = 0.2
    delta = w + tooth + gap
    aero_tower = pipe(d=d, h=h, w=w, tooth=2, socket_h=socket_h)
    (cup_s, cup, cup_outline) = cup_con(d1=cup_d-cup_h//10,d2=cup_d, w=w, height=cup_h, inlet_delta=5)
    outline = cylinder(d=d, h=d, segments=100)
    holes = spin_around_tower(cup_outline, count=pipe_count, alpha=alpha)
    sockets = spin_around_tower(
        translate((0, 0, h/2+w))(cup_s), h=d, count=pipe_count, alpha=alpha)

    holes = translate((0, 0, -h/2))(holes)
    sockets = translate((0, 0, -h/2))(sockets)
    aero_tower = aero_tower - holes + sockets
    aero_tower *= outline

    return (aero_tower, cup)


(tower, cup ) = aero_tower(pipe_count=5)
render(tower, outfile='./outfiles/aero_tower.stl')
render(cup, outfile='./outfiles/plant_cup.stl')


In [ ]:
from solid import cylinder
from solid.utils import up
from stackable_pipe import connection, ring, two_sided_connection 
from viewscad import Renderer

r = Renderer()
r.width = 700
r.height = 700

render = r.render

def tank(width=180, height=180, wall=2, socket_h=20, tooth=2):
    body = ring(d1=width, h=height, w=wall)
    bottom = cylinder(d=width, h=wall, segments=100)
    (_, socket) = connection(d=width, h=socket_h,
                             wall=wall, tooth_depth=tooth, tooth_height=tooth*2)

    return bottom + body + up(height-socket_h)(socket)


render(two_sided_connection(), outfile='./outfiles/aero_tower_tank_adapter.stl')
render(tank(), outfile='./outfiles/aero_tower_tank.stl')


In [14]:

from stackable_pipe import pipe, ring, connection
from solid import cylinder, translate, rotate, cube
from solid.utils import up, forward
from viewscad import Renderer

r = Renderer()
r.width = 700
r.height = 700

render = r.render


def fog_well(inner_d=50, wall=2, well_depth=60, adapter_h=10, hole_h=20, tube_d=8, cable_d=15, tooth=2):
    outer_d = inner_d+wall*2+tooth*2
    total_h = well_depth+wall+adapter_h+hole_h
    well = ring(d1=outer_d, w=wall, h=total_h)
    base = cylinder(h=wall, d=outer_d, segments=100)

    hole = cylinder(d=hole_h, segments=3, h=outer_d)
    holes = [rotate((0, 270, z))(hole) for z in range(0, 360, 360//5)]

    (adapter, socket) = connection(wall=wall, d=outer_d,
                                   h=adapter_h, tooth_depth=tooth, tooth_height=tooth*2, gap=0.1)

    fog_well = base + well\
        - up(well_depth)(holes)\
        + up(well_depth+hole_h)(socket)
    lid = adapter\
        + cylinder(d=inner_d, h=adapter_h, segments=100)\
        - cylinder(d=cable_d, h=adapter_h, segments=100)\
        - forward((inner_d-cable_d)/2)(
            cylinder(d=tube_d, h=adapter_h, segments=100)
        )

    return (fog_well, lid)


def electronics_mount(d=150, hole_d=67, strength=10, bb_mount_h=16, bb_w=83, bb_d=10):
    mock = ring(h=1, d1=d, w=(d-hole_d)/2)

    bb_mount_delta = (hole_d + bb_d)/2
    bb_mount = up(bb_mount_h/2)(
        cube((bb_d+strength, bb_w+2, bb_mount_h), center=True)
        - cube((bb_d, bb_w, bb_mount_h), center=True)
    )
    bb_mount = translate((-bb_mount_delta, 0, 0))(bb_mount)
    bb_mount *= ring(r1=bb_mount_delta + strength, h=bb_mount_h, w=strength)


    return mock + bb_mount


def tower_head(d=160, w=2, tooth=2, socket_h=20, h=50, gap=0.1, fogger_d=50, fw_depth=60, fw_socket_h=10, fw_hole_h=20):
    delta_r = w + tooth + gap
    body = pipe(d=d, w=w, tooth=tooth, socket_h=socket_h, h=h)

    fog_well_d = fogger_d + (tooth + w)*2 + (tooth + w + gap)*2

    (adapter, socket) = connection(d=fog_well_d,
                                   tooth_depth=tooth, tooth_height=tooth*2, h=socket_h, gap=gap)
    (well, lid) = fog_well(inner_d=fogger_d, wall=w, tooth=tooth,
                           well_depth=fw_depth, adapter_h=fw_socket_h, hole_h=fw_hole_h)

    well_h = fw_depth + fw_socket_h + fw_hole_h + w
    well = well + up(well_h-socket_h)(adapter)

    base = cylinder(r=d/2-delta_r, h=w, segments=100)\
        - cylinder(d=fog_well_d, h=w, segments=100)\
        + socket

    head = body + base

    return (head, well, lid)


render(electronics_mount())


# (head, well, lid) = tower_head()

# render(head, outfile='outfiles/aero_tower_head.stl')
# render(well, outfile='outfiles/fog_well.stl')
# render(lid, outfile='outfiles/fog_well_lid.stl')


Geometries in cache: 6
Geometry cache size in bytes: 59664
CGAL Polyhedrons in cache: 9
CGAL cache size in bytes: 2959536
Total rendering time: 0:00:05.931
   Top level object is a 3D object:
   Simple:        yes
   Vertices:      474
   Halfedges:    1422
   Edges:         711
   Halffacets:    484
   Facets:        242
   Volumes:         2
